# オープンデータとデータ取得

## オープンデータ

### オープンデータとは

- [非営利組織Open Knowledge Foundation](https://okfn.org/en/)
- [OPEN DATA HANDBOOK](https://opendatahandbook.org/guide/ja/what-is-open-data/)
- [full Open Definition](http://opendefinition.org/okd/)
- [オープンデータ基本指針](https://www.digital.go.jp/assets/contents/node/basic_page/field_ref_resources/f7fde41d-ffca-4b2a-9b25-94b8a701a037/f1e42cee/20240705_resources_data_guideline_01.pdf)
- [官民データ活用推進基本法](https://laws.e-gov.go.jp/law/428AC1000000103)
- [デジタル社会推進会](https://www.digital.go.jp/councils/social-promotion)
- [政府CIO ポータル](https://cio.go.jp/policy-opendata)
- [デジタル庁](https://www.digital.go.jp/resources/open_data)

### オープンデータの公開レベル

[オープンデータのための5 つ星スキーム](https://5stardata.info/ja/)

#### RDFとLOD

- [W3C](https://www.w3.org/)
- [e-Stat 統計LOD](https://data.e-stat.go.jp/lodw/outline/abstraction)

## データ取得

### CSV等ファイルダウンロード

[EDINET](https://disclosure2.edinet-fsa.go.jp/week0010.aspx)


In [1]:
from pathlib import Path
import pandas as pd
current_dir = Path.cwd()
data_path = (current_dir / "data" / "ch09").resolve()
csv_data = pd.read_csv(data_path / "EdinetcodeDlInfo.csv", encoding="cp932", skiprows=1)

#### Google Colabの場合、GitHubの"EdinetcodeDlInfo.csv"を直接読み込む
from urllib.parse import urljoin, quote
import pandas as pd
github_path = "https://raw.githubusercontent.com/python-opendata-analysis/python-opendata-analysis-book/main/code/data/ch09/"
csv_url = urljoin(github_path, "EdinetcodeDlInfo.csv")
csv_data = pd.read_csv(csv_url, encoding="cp932", skiprows=1)

### APIとPythonのRequestsライブラリ

#### REST API

#### Pythonライブラリのrequests

- [urllib.request](https://docs.python.org/ja/3/library/urllib.request.html)
- [Requests](https://requests.readthedocs.io/en/latest/)
- [Requests(GitHub)](https://github.com/psf/requests/blob/main/src/requests/sessions.py)


In [2]:
import requests
url = "https://dashboard.e-stat.go.jp/api/1.0/Json/getData"
params = {
    "IndicatorCode": "0704010101000010000", 
    "TimeFrom": "20200100",
}
res = requests.get(url=url, params=params)

In [3]:
res.status_code

200

In [4]:
res.encoding

'UTF-8'

In [ ]:
res.text[:100] 

'{"GET_STATS":{"RESULT":{"status":"0","errorMsg":"\\u6B63\\u5E38\\u306B\\u7D42\\u4E86\\u3057\\u307E\\u3057\\u305F\\u3002","date":"Tue Jan 27 14:32:28 JST 2026"},"PARAMETER":{"lang":"JP","indicatorCode":["0704010101000010000"],"timeFrom":"20200100"},"STATISTICAL_DATA":{"RESULT_INF":{"TOTAL_NUMBER":"3763"},"TABLE_INF":{"STAT_NAME":[{"@code":"00200561","$":"\\u5BB6\\u8A08\\u8ABF\\u67FB"}]},"DATA_INF":{"DATA_OBJ":[{"VALUE":{"@indicator":"0704010101000010000","@unit":"116","@stat":"00200561","@regionCode":"00000","@time":"20200100","@cycle":"1","@regionRank":"2","@isSeasonal":"1","@isProvisional":"0","$":"287173"}},{"VALUE":{"@indicator":"0704010101000010000","@unit":"116","@stat":"00200561","@regionCode":"01100","@time":"20200100","@cycle":"1","@regionRank":"4","@isSeasonal":"1","@isProvisional":"0","$":"274960"}},{"VALUE":{"@indicator":"0704010101000010000","@unit":"116","@stat":"00200561","@regionCode":"02201","@time":"20200100","@cycle":"1","@regionRank":"4","@isSeasonal":"1","@isProvis

In [12]:
# 1. APIから返ってきたレスポンス（生のデータ）を、Pythonの辞書形式（JSON）として読み込む
# これにより、データの各項目に名前（キー）を使ってアクセスできるようになります
json_data = res.json()

# 2. 読み込んだデータの「一番上の階層」にどんな名前の項目（キー）があるかを確認する
# 複雑なデータ構造の全体像をつかむための最初の一歩です
json_data.keys()

dict_keys(['GET_STATS'])

In [ ]:
# GET_STATSの下にあるキーを確認する
json_data['GET_STATS'].keys()

dict_keys(['RESULT', 'PARAMETER', 'STATISTICAL_DATA'])

In [ ]:
# GET_STATS -> STATISTICAL_DATA の下にあるキーを確認する
json_data['GET_STATS']['STATISTICAL_DATA'].keys()

dict_keys(['RESULT_INF', 'TABLE_INF', 'DATA_INF'])

In [ ]:
# GET_STATS -> STATISTICAL_DATA -> DATA_INF の下にあるキーを確認する
json_data['GET_STATS']['STATISTICAL_DATA']['DATA_INF'].keys()

dict_keys(['DATA_OBJ'])

#### なぜ `.keys()` を使うのか？
Pythonの「辞書（dict）」という形式では、`.keys()` を使うことで、その箱の中に**「どんな名前の見出し（ラベル）」**が入っているかだけを一覧表示できます。

いきなり `json_data["GET_STATS"]["STATISTICAL_DATA"]` 全体を表示してしまうと、中身の大量のデータまで表示されてしまって、構造が逆に見えなくなります。

「見出し（キー）だけを見て、一歩ずつ進む」。 これが、不親切な解説に頼らず自力でデータ構造を読み解くための「プロのやり方」です。ぜひ試してみてください。

In [13]:
# 複雑な階層を順に掘り進み、実際のデータ本体にアクセスして、最初の2件だけを表示
# 階層：統計情報の取得（GET_STATS） > 統計データ本体 > データ情報 > オブジェクトのリスト[:先頭2件]
json_data["GET_STATS"]["STATISTICAL_DATA"]["DATA_INF"]["DATA_OBJ"][:2]

[{'VALUE': {'@indicator': '0704010101000010000',
   '@unit': '116',
   '@stat': '00200561',
   '@regionCode': '00000',
   '@time': '20200100',
   '@cycle': '1',
   '@regionRank': '2',
   '@isSeasonal': '1',
   '@isProvisional': '0',
   '$': '287173'}},
 {'VALUE': {'@indicator': '0704010101000010000',
   '@unit': '116',
   '@stat': '00200561',
   '@regionCode': '01100',
   '@time': '20200100',
   '@cycle': '1',
   '@regionRank': '4',
   '@isSeasonal': '1',
   '@isProvisional': '0',
   '$': '274960'}}]

In [24]:
# 1. サーバーとの「一連のやり取り（セッション）」を開始する窓口をひらく
# withを使うことで、終わったあとに自動で接続を片付けて（閉じて）くれます
with requests.Session() as session:
    
    # 2. その窓口（session）を通じて、データをリクエストする
    # method="GET"（データを取得）、url（住所）、params（条件）を指定
    res = session.request(method="GET", url=url, params=params)
    
    # 3. 取得したデータの深い階層にある「数値データ（先頭2件）」を表示する
    print(res.json()["GET_STATS"]["STATISTICAL_DATA"]["DATA_INF"]["DATA_OBJ"][:2])

[{'VALUE': {'@indicator': '0704010101000010000', '@unit': '116', '@stat': '00200561', '@regionCode': '00000', '@time': '20200100', '@cycle': '1', '@regionRank': '2', '@isSeasonal': '1', '@isProvisional': '0', '$': '287173'}}, {'VALUE': {'@indicator': '0704010101000010000', '@unit': '116', '@stat': '00200561', '@regionCode': '01100', '@time': '20200100', '@cycle': '1', '@regionRank': '4', '@isSeasonal': '1', '@isProvisional': '0', '$': '274960'}}]


#### 何が変わったのか？（なぜこれを使うのか）
1. 効率が良い（Session）: これまでの `requests.get()` は、1回データを取るたびに「こんにちは（接続）」「さようなら（切断）」をしていました。`Session` を使うと、同じ相手なら接続を繋ぎっぱなしにできるため、複数回データを取る場合にスピードが速くなります。
2. 安全（with構文）: `with` という書き方を使うと、**「中身の処理が終わったら、接続を確実に閉じる」**という後始末をPythonが自動で保証してくれます。ブラウザのタブを出しっぱなしにするのではなく、使い終わったら勝手に閉じてくれるイメージです。
3. 汎用性が高い (session.request): `session.request(method="GET", ...)` という書き方は、「今回はGETだけど、次はデータ送信のPOSTに変更する」といった対応がしやすく、より丁寧な書き方とされています。

一言で言うと： 「単発で注文する」のではなく、**「お店との専用カウンターを開設して、そこで丁寧に注文し、終わったら自動で店仕舞いする」**という、少しプロ寄りの丁寧な書き方になりました。

#### APIキーの設定


In [ ]:
api_key = "ここにAPIキーを書く" # この方法はGitHub連携をするなら良くない。環境変数もしくは.envファイルで管理する

data_url = "https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData"
data_params = {
    "appId": api_key,
    "statsDataId": "0003000795",
    "limit": 100,
}
data_res = requests.get(url=data_url, params=data_params)
data_res.json()["GET_STATS_DATA"]["STATISTICAL_DATA"]["DATA_INF"]["VALUE"]

##### APIキーを環境変数に設定する


In [ ]:
import os
api_key = os.getenv("ESTAT_APP_ID")

##### .envファイルに設定する


In [29]:
import os
from dotenv import load_dotenv
# .envファイルの内容を読み込む
load_dotenv()
# 環境変数からAPIキーを取得
api_key = os.getenv("ESTAT_APP_ID")

In [ ]:
# config.envファイルの内容を読み込む
#load_dotenv(dotenv_path="config.env")
#api_key = os.getenv("ESTAT_APP_ID")

##### ColabでAPIキーを設定する

In [ ]:
#from google.colab import userdata
#api_key = userdata.get("ESTAT_APP_ID")

#### OpenAPIとSwagger

- [OpenAPI](https://www.openapis.org/)
- [Swagger](https://swagger.io/)
- [gBizINFO REST API](https://info.gbiz.go.jp/hojin/swagger-ui/index.html)

#### LODとSPARQL

- [e-Stat 統計LOD](https://data.e-stat.go.jp/lodw/)
- [SPARQL](https://www.w3.org/TR/sparql11-query/)
- [e-Stat 統計LOD「統計表の基本的なデータ構造」](https://data.e-stat.go.jp/lodw/outline/abstraction#1-1-2-1-2)
- [SPARQLWrapper](https://sparqlwrapper.readthedocs.io/en/latest/index.html)
- [e-Stat 統計LOD「統計LOD の基本的な使い方」](https://data.e-stat.go.jp/lodw/lodw/index.php/guidelines/howto)


In [ ]:
#!pip install sparqlwrapper

In [31]:
from SPARQLWrapper import SPARQLWrapper, JSON
SPARQL_ENDPOINT = "http://data.e-stat.go.jp/lod/sparql/alldata/query"
sparql = SPARQLWrapper(SPARQL_ENDPOINT)
sparql.setReturnFormat(JSON)

年と人口のみを取得

In [32]:
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sdmx-dimension: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX estat-measure: <http://data.e-stat.go.jp/lod/ontology/measure/>
PREFIX cd-dimension: <http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/>
PREFIX cd-code: <http://data.e-stat.go.jp/lod/ontology/crossDomain/code/>
PREFIX g00200521-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/dimension/2010/>
PREFIX g00200521-code-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/code/2010/>
select  ?year ?population
where {
      ?s estat-measure:population ?population ;
         sdmx-dimension:refArea / rdfs:label "新宿区"@ja ;
         cd-dimension:timePeriod ?year ;
         cd-dimension:sex cd-code:sex-all ;
         cd-dimension:nationality cd-code:nationality-japan ;
         g00200521-dimension-2010:area g00200521-code-2010:area-all ;
         cd-dimension:age cd-code:age-all .
}
    """
)

In [33]:
ret = sparql.queryAndConvert()
ret

{'head': {'vars': ['YEAR', 'POPULATION'], 'links': []},
 'results': {'bindings': [{'YEAR': {'type': 'literal',
     'value': '2015',
     'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'},
    'POPULATION': {'type': 'literal',
     'value': '300013',
     'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}},
   {'YEAR': {'type': 'literal',
     'value': '2010',
     'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'},
    'POPULATION': {'type': 'literal',
     'value': '286398',
     'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}}]}}

主語（?s）も含めて取得

In [34]:
sparql.setQuery("""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX sdmx-dimension: <http://purl.org/linked-data/sdmx/2009/dimension#>
PREFIX estat-measure: <http://data.e-stat.go.jp/lod/ontology/measure/>
PREFIX cd-dimension: <http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/>
PREFIX cd-code: <http://data.e-stat.go.jp/lod/ontology/crossDomain/code/>
PREFIX g00200521-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/dimension/2010/>
PREFIX g00200521-code-2010:<http://data.e-stat.go.jp/lod/ontology/g00200521/code/2010/>
select  ?s ?year ?population
where {
      ?s estat-measure:population ?population ;
         sdmx-dimension:refArea / rdfs:label "新宿区"@ja ;
         cd-dimension:timePeriod ?year ;
         cd-dimension:sex cd-code:sex-all ;
         cd-dimension:nationality cd-code:nationality-japan ;
         g00200521-dimension-2010:area g00200521-code-2010:area-all ;
         cd-dimension:age cd-code:age-all .
}
    """
)
ret = sparql.queryAndConvert()
ret

{'head': {'vars': ['S', 'YEAR', 'POPULATION'], 'links': []},
 'results': {'bindings': [{'S': {'type': 'uri',
     'value': 'http://data.e-stat.go.jp/lod/dataset/g00200521/d0003148521/obsKVGVPVSO5CVMX6TNHAEKKLTXZAMKEGMH'},
    'YEAR': {'type': 'literal',
     'value': '2015',
     'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'},
    'POPULATION': {'type': 'literal',
     'value': '300013',
     'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}},
   {'S': {'type': 'uri',
     'value': 'http://data.e-stat.go.jp/lod/dataset/g00200521/d0003041389/obsMAQ75S54YTCW3432OAKUOOXIECY7M6OO'},
    'YEAR': {'type': 'literal',
     'value': '2010',
     'datatype': 'http://www.w3.org/2001/XMLSchema#gYear'},
    'POPULATION': {'type': 'literal',
     'value': '286398',
     'datatype': 'http://www.w3.org/2001/XMLSchema#decimal'}}]}}

In [35]:
ret.keys()

dict_keys(['head', 'results'])

In [36]:
ret["head"]

{'vars': ['S', 'YEAR', 'POPULATION'], 'links': []}

In [37]:
ret["results"].keys()

dict_keys(['bindings'])

In [38]:
sparql_df = pd.json_normalize(ret, record_path=["results", "bindings"])
sparql_df

,S.type,S.value,YEAR.type,YEAR.value,YEAR.datatype,POPULATION.type,POPULATION.value,POPULATION.datatype
0,uri,http://data.e-stat.go.jp/lod/dataset/g00200521...,literal,2015,http://www.w3.org/2001/XMLSchema#gYear,literal,300013,http://www.w3.org/2001/XMLSchema#decimal
1,uri,http://data.e-stat.go.jp/lod/dataset/g00200521...,literal,2010,http://www.w3.org/2001/XMLSchema#gYear,literal,286398,http://www.w3.org/2001/XMLSchema#decimal


In [39]:
sparql_df.columns = pd.MultiIndex.from_tuples([tuple(col.split(".")) for col in sparql_df.columns])
sparql_df

S                                                        YEAR        \
  type                                              value     type value   
0  uri  http://data.e-stat.go.jp/lod/dataset/g00200521...  literal  2015   
1  uri  http://data.e-stat.go.jp/lod/dataset/g00200521...  literal  2010   

                                          POPULATION          \
                                 datatype       type   value   
0  http://www.w3.org/2001/XMLSchema#gYear    literal  300013   
1  http://www.w3.org/2001/XMLSchema#gYear    literal  286398   

                                             
                                   datatype  
0  http://www.w3.org/2001/XMLSchema#decimal  
1  http://www.w3.org/2001/XMLSchema#decimal